# 타이타닉_최종

## . 

### 필요한 라이브러리 설치

In [ ]:
#데이터 호출
import pandas as pd
import numpy as np
import random as rnd

In [ ]:
# 시각화
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

In [ ]:
# 결측치 제거
from sklearn.impute import SimpleImputer


In [ ]:
# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

### 타이타닉 데이터 불러오기

In [ ]:
# train 데이터 불러오기
train = pd.read_csv('C:/Users/Edward/Desktop/PythonWorkspace/Titanic/train.csv')
train_no = train
train_normal = train
# 확인
train.head(2)

In [ ]:
print(train.shape)
print(train_no.shape)
print(train_normal.shape)

In [ ]:
# test 데이터 불러오기
test = pd.read_csv('C:/Users/Edward/Desktop/PythonWorkspace/Titanic/test.csv')
test_no = test
test_normal = test
# 확인
test.head(2)

In [ ]:
print(test.shape)
print(test_no.shape)
print(test_normal.shape)

In [ ]:
# 모델 적용 일관성을 위해, train과 test를 합친 combine 생성
combine = [train,test]
combine_no = [train_no, test_no]
combine_normal = [train_normal, test_normal]

### 변수 확인

In [ ]:
print(train.columns.values)
print(train_no.columns.values)
print(train_normal.columns.values)

### 변수 설명

|변수|설명|
|------|---|
|survival|Survival (0 = 사망; 1 = 생존)|
|pclass|Passenger Class (1 = 1등석; 2 = 2등석; 3 = 3등석)|
|name|이름|
|sex|성별|
|age|나이|
|sibsp|동반한 형제자매(Sibling)와 배우자(Spouse)의 수|
|parch|동반한 부모(Parents)와 자식(Children)의 수|
|ticket|티캣 번호|
|fare|품삯|
|cabin|객실 번호|
|embarked|탑승한 항구 (C = Cherbourg; Q = Queenstown; S = Southampton)|

### 데이터 확인

In [ ]:
train.head()

In [ ]:
train.tail()

#### 데이터 정보

In [ ]:
# 훈련 셋 정보
train.info()

In [ ]:
# 검증 셋 정보
test.info()

In [ ]:
# 훈련 자료 int, float 변수 통계치
train.describe()

총 샘플 수 : 891
샘플 내 생존율 : 38.38% (mean의 survived=0.383838)

In [ ]:
# 훈련 자료 object 변수 통계치
train.describe(include=['O'])

Embarked 최빈값은 "S"

## 데이터 분석

### Pclass별 생존율

In [ ]:
train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)
#groupby에 as_index를 False로 해야 Pclass를 index로 사용하지 않음
# ascending : 오름차순

객실 등급이 높을수록 생존율이 높음

### Sex별 생존율

In [ ]:
train[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

여성의 생존율이 매우 높음

### SibSp별 생존율

In [ ]:
train[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

### Parch별 생존율

In [ ]:
train[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

### Age별 생존율

In [ ]:
# 열(col)을 생존 여부로 나눔
g = sns.FacetGrid(train, col='Survived')
# 히스토그램으로 시각화, 연령의 분포를 확인, 히스토그램 bin을 20개로 설정
g.map(plt.hist, 'Age', bins=20)

0-4세와 16-38세의 생존율이 높음

### Pclass와 Survived간의 Age 분포

In [ ]:
# 열을 생존 여부, 행(row)과 색깔(hue)을 객실 등급으로 나눔, width = height * aspect
grid = sns.FacetGrid(train, col='Survived', row='Pclass', hue="Pclass", height=2.2, aspect=1.6)

grid.map(plt.hist, 'Age', alpha=.5, bins=20) # 투명도(alpha): 0.5

# 범례 추가
grid.add_legend()

1등석의 경우 생존율이 좀 높은 경향을 보임  
2등석의 경우 영아 생존율이 높음  
3등석의 경우 승객 비중은 가장 많지만, 생존율은 가장 낮음

### Pclass별 Sex 분포

In [ ]:
grid = sns.FacetGrid(train, row='Pclass', col='Sex', hue="Pclass", height=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend()

### Embarked와 Pclass에 따른 Survived 분포

In [ ]:
grid = sns.FacetGrid(train, row='Embarked', height=2.2, aspect=1.6)

# Pointplot으로 시각화, x: 객실 등급, y: 생존 여부, 색깔: 성별, x축 순서: [1, 2, 3], 색깔 순서: [남성, 여성]
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette='deep', order = [1, 2, 3], hue_order = ["male", "female"])

grid.add_legend()

승선지가 C와 Q의 경우, 남성이 3등석 승객일 때, 2등석보다 생존율이 높음  
승선지가 C의 경우, 여성이 2등석 승객일 때, 1등석보다 생존율이 높음

### Embarked, Survived, Sex에 따른 Fare

In [ ]:
grid = sns.FacetGrid(train, row='Embarked', col='Survived', height=2.2, aspect=1.6)

# 바그래프로 시각화, x: 성별, y: 요금, 
grid.map(sns.barplot, 'Sex', 'Fare', order=["male","female"])

### 사용하지 않는 변수 제거 1

In [ ]:
print("Before", train.shape, test.shape)

train = train.drop(['Ticket', 'PassengerId'], axis=1)
test = test.drop(['Ticket'], axis=1)
combine = [train, test]

print("After", train.shape, test.shape)

In [ ]:
print("Before", train_no.shape, test_no.shape)

train_no = train_no.drop(['Ticket', 'PassengerId'], axis=1)
test_no = test_no.drop(['Ticket'], axis=1)
combine_no = [train_no, test_no]

print("After", train_no.shape, test_no.shape)

In [ ]:
print("Before", train_normal.shape, test_normal.shape)

train_normal = train_normal.drop(['Ticket', 'PassengerId'], axis=1)
test_normal = test_normal.drop(['Ticket'], axis=1)
combine_normal = [train_normal, test_normal]

print("After", train_normal.shape, test_normal.shape)

### Name에서 Title 추출

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(train['Title'], train['Sex'])

In [ ]:
for dataset in combine_normal:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(train_normal['Title'], train_normal['Sex'])

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Noble')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

In [ ]:
for dataset in combine_normal:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Noble')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train_normal[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

In [ ]:
train.head()

In [ ]:
train_normal.head()

### 변수를 숫자형 변수로 변환

#### Sex

In [ ]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train.head()

In [ ]:
for dataset in combine_no:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train_no.head()

In [ ]:
for dataset in combine_normal:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train_normal.head()

#### Title

In [ ]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train.head()

In [ ]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine_normal:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train_normal.head()

### 사용하지 않는 변수 제거 2

In [ ]:
train = train.drop(['Name'], axis=1)
test= test.drop(['Name'], axis=1)
combine = [train, test]
train.shape, test.shape

In [ ]:
train_no = train_no.drop(['Name'], axis=1)
test_no = test_no.drop(['Name'], axis=1)
combine_no = [train_no, test_no]
train_no.shape, test_no.shape

In [ ]:
train_normal = train_normal.drop(['Name'], axis=1)
test_normal = test_normal.drop(['Name'], axis=1)
combine_normal = [train_normal, test_normal]
train_normal.shape, test_normal.shape

### 결측치 처리

#### 결측치 확인

In [ ]:
train.isna().sum()

In [ ]:
train_no.isna().sum()

In [ ]:
train_normal.isna().sum()

#### 결측치 시각화

In [ ]:
msno.matrix(train)
plt.show()

In [ ]:
msno.bar(train)
plt.show()

In [ ]:
msno.matrix(train_no)
plt.show

In [ ]:
msno.bar(train_no)
plt.show

In [ ]:
msno.matrix(train_normal)
plt.show

In [ ]:
msno.bar(train_no)
plt.show

#### Embarked 결측치 제거

In [ ]:
# 대상을 리스트로 선언
imputer1_list = ['Embarked']

# 선언하고 fit_transform
imputer1 = SimpleImputer(strategy = 'most_frequent') # 최빈값을 이용한 결측치를 채우는 함수 사용
train[imputer1_list] = imputer1.fit_transform(train[imputer1_list])
train.isna().sum()

In [ ]:
train_no[imputer1_list] = imputer1.fit_transform(train_no[imputer1_list])
train_no.isna().sum()

In [ ]:
train_normal[imputer1_list] = imputer1.fit_transform(train_normal[imputer1_list])
train_normal.isna().sum()

### Cabin 결측치 제거

In [ ]:
for dataset in combine:
    dataset['Cabin'] = dataset['Cabin'].fillna('N')
    
train.isna().sum()

In [ ]:
for dataset in combine_no:
    dataset['Cabin'] = dataset['Cabin'].fillna('N')
    
train_no.isna().sum()

In [ ]:
for dataset in combine_normal:
    dataset['Cabin'] = dataset['Cabin'].fillna('N')
    
train_normal.isna().sum()

In [ ]:
train.Cabin.unique()

In [ ]:
train_no.Cabin.unique()

In [ ]:
train_normal.Cabin.unique()

In [ ]:
for dataset in combine:
    dataset['Cabin_class'] = dataset['Cabin']
    dataset['Cabin_class'] = dataset.Cabin.str.extract('([A-Za-z]+)', expand=False)

train.Cabin_class.unique()

In [ ]:
for dataset in combine_normal:
    dataset['Cabin_class'] = dataset['Cabin']
    dataset['Cabin_class'] = dataset.Cabin.str.extract('([A-Za-z]+)', expand=False)

train_normal.Cabin_class.unique()

In [ ]:
train[["Cabin_class", "Survived"]].groupby(['Cabin_class'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
Cabin_mapping = {"T": 0, "N": 1, "A": 2, "G": 3, "C": 4, "F": 5, "B": 6, "E": 7, "D": 8}
for dataset in combine:
    dataset['Cabin_class'] = dataset['Cabin_class'].map(Cabin_mapping)
    dataset['Cabin_class'] = dataset['Cabin_class'].fillna(0)

train.head()

In [ ]:
Cabin_mapping = {"T": 0, "N": 1, "A": 2, "G": 3, "C": 4, "F": 5, "B": 6, "E": 7, "D": 8}
for dataset in combine_normal:
    dataset['Cabin_class'] = dataset['Cabin_class'].map(Cabin_mapping)
    dataset['Cabin_class'] = dataset['Cabin_class'].fillna(0)

train_normal.head()

### 사용하지 않는 변수 제거 3

In [ ]:
train = train.drop(['Cabin'], axis=1)
test = test.drop(['Cabin'], axis=1)
combine = [train, test]
train.shape, test.shape

In [ ]:
train_no = train_no.drop(['Cabin'], axis=1)
test_no = test_no.drop(['Cabin'], axis=1)
combine_no = [train_no, test_no]
train_no.shape, test_no.shape

In [ ]:
train_normal = train_normal.drop(['Cabin'], axis=1)
test_normal = test_normal.drop(['Cabin'], axis=1)
combine_normal = [train_normal, test_normal]
train_normal.shape, test_normal.shape

### Age 결측치 제거

In [ ]:
guess_ages = np.zeros((2,3)) # 0으로 가득한 2*3 Array 생성
for dataset in combine:
    for i in range(0, 2): # i를 0부터 2까지 반복
        for j in range(0, 3): # j를 0부터 3까지 반복
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna() # 여기서 부터 작동원리 이해 안됨
            # 위에서 guess_ages사이즈를 [2,3]으로 잡아뒀으므로 j의 범위도 이를 따름
            
            age_guess = guess_df.median()

            # age의 random값의 소수점을 .5에 가깝도록 변형
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

train.isna().sum()

In [ ]:
for dataset in combine_no:
    dataset['Age'] = dataset['Age'].fillna(dataset['Age'].mean())
    dataset['Age'] = dataset['Age'].fillna(0)
train_no.isna().sum()

In [ ]:
for dataset in combine_normal:
    dataset['Age'] = dataset['Age'].fillna(dataset['Age'].mode())
    dataset['Age'] = dataset['Age'].fillna(0)
train_normal.isna().sum()

### Age를 범주형으로 변환

In [ ]:
train['AgeBand'] = pd.cut(train['Age'], 5)
# 임의로 5개 그룹을 지정
train[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

생성된 AgeBand을 통해 Age를 범주형으로 변환 후, AgeBand 제거

In [ ]:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4
train = train.drop(['AgeBand'], axis=1)
combine = [train, test]
train.head()

### SibSp와 Parch를 활용하여 FamilySize 변수 생성

In [ ]:
for dataset in combine:
    dataset['Familysize'] = dataset['SibSp'] + dataset['Parch'] + 1

train[['Familysize', 'Survived']].groupby(['Familysize'], as_index=False).mean().sort_values(by='Familysize', ascending=True)

Familysize 1은 가족을 동반하지 않음을 의미

In [ ]:
grid = sns.FacetGrid(train, height=2.2, aspect=1.6)

# Pointplot으로 시각화, x: 객실 등급, y: 생존 여부, 색깔: 성별, x축 순서: [1, 2, 3], 색깔 순서: [남성, 여성]
grid.map(sns.lineplot, 'Familysize', 'Survived', palette='deep', markers=True)

### Age와 Pclass 값을 곱한 Age*Pclass 변수 생성

In [ ]:
for dataset in combine:
    dataset['Age*Pclass'] = dataset.Age * dataset.Pclass

train.loc[:, ['Age*Pclass', 'Age', 'Pclass']].head(10)

### Embarked 범주형 변수로 변환

In [ ]:
train[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train.head()

In [ ]:
for dataset in combine_no:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train_no.head()

In [ ]:
for dataset in combine_normal:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train_normal.head()

### Fare를 중앙값으로 대체

In [ ]:
test['Fare'].fillna(test['Fare'].dropna().median(), inplace=True)
test.head()

In [ ]:
test_no['Fare'].fillna(test_no['Fare'].dropna().median(), inplace=True)
test_no.head()

In [ ]:
test_normal['Fare'].fillna(test_normal['Fare'].dropna().median(), inplace=True)
test_normal.head()

In [ ]:
train['FareBand'] = pd.qcut(train['Fare'], 4)
train[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

In [ ]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train = train.drop(['FareBand'], axis=1)
combine = [train, test]
    
train.head(10)

## 데이터 준비

In [1]:
""" # 목적 변수 제거
X_train = train.drop("Survived", axis=1)
#목적 변수 역할
Y_train = train["Survived"]
#예측 대상 데이터 셋
X_test = test.drop("PassengerId", axis=1).copy()
Y_test = test['Survived']
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape """

' # 목적 변수 제거\nX_train = train.drop("Survived", axis=1)\n#목적 변수 역할\nY_train = train["Survived"]\n#예측 대상 데이터 셋\nX_test = test.drop("PassengerId", axis=1).copy()\nY_test = test[\'Survived\']\nX_train.shape, Y_train.shape, X_test.shape, Y_test.shape '

In [ ]:
""" # 목적 변수 제거
X_train_no = train_no.drop("Survived", axis=1)
#목적 변수 역할
Y_train_no = train_no["Survived"]
#예측 대상 데이터 셋
X_test_no  = test_no.drop("PassengerId", axis=1).copy()
Y_test_no = test_no['Survived']
X_train_no.shape, Y_train_no.shape, X_test_no.shape, Y_test_no.shape """

In [ ]:
""" # 목적 변수 제거
X_train_normal = train_normal.drop("Survived", axis=1)
#목적 변수 역할
Y_train_normal = train_normal["Survived"]
#예측 대상 데이터 셋
X_test_normal  = test_normal.drop("PassengerId", axis=1).copy()
Y_test_normal = test_no['Survived']
X_train_normal.shape, Y_train_normal.shape, X_test_normal.shape, Y_test_normal.shape """

In [ ]:
train.info()

In [ ]:
train_no.info()

In [ ]:
test_no.info()

In [ ]:
train_normal.info()

### Logistic Regression

In [ ]:
# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

In [ ]:
coeff_df = pd.DataFrame(train.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

In [ ]:
# Logistic Regression

logreg_no = LogisticRegression()
logreg_no.fit(X_train_no, Y_train_no)
Y_pred_no = logreg_no.predict(X_test_no)
acc_log_no = round(logreg_no.score(X_train_no, Y_train_no) * 100, 2)
acc_log_no

In [ ]:
coeff_df_no = pd.DataFrame(train_no.columns.delete(0))
coeff_df_no.columns = ['Feature']
coeff_df_no["Correlation"] = pd.Series(logreg_no.coef_[0])

coeff_df_no.sort_values(by='Correlation', ascending=False)

In [ ]:
# Logistic Regression

logreg_normal = LogisticRegression()
logreg_normal.fit(X_train_normal, Y_train_normal)
Y_pred_normal = logreg_normal.predict(X_test_normal)
acc_log_normal = round(logreg_normal.score(X_train_normal, Y_train_normal) * 100, 2)
acc_log_normal

In [ ]:
coeff_df_normal = pd.DataFrame(train_normal.columns.delete(0))
coeff_df_normal.columns = ['Feature']
coeff_df_normal["Correlation"] = pd.Series(logreg_normal.coef_[0])

coeff_df_normal.sort_values(by='Correlation', ascending=False)

생존 여부 예측에 영향력이 큰 변수는 성별(Sex)

### SCM (Support Vector Machines)

In [ ]:
# Support Vector Machines

svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

In [ ]:
svc_no = SVC()
svc_no.fit(X_train_no, Y_train_no)
Y_pred_no = svc_no.predict(X_test_no)
acc_svc_no = round(svc_no.score(X_train_no, Y_train_no) * 100, 2)
acc_svc_no

In [ ]:
svc_normal = SVC()
svc_normal.fit(X_train_normal, Y_train_normal)
Y_pred_normal = svc_normal.predict(X_test_normal)
acc_svc_normal = round(svc_normal.score(X_train_normal, Y_train_normal) * 100, 2)
acc_svc_normal

### KNN (K Nearest Neighberhood)

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

In [ ]:
knn_no = KNeighborsClassifier(n_neighbors = 3)
knn_no.fit(X_train_no, Y_train_no)
Y_pred_no = knn_no.predict(X_test_no)
acc_knn_no = round(knn_no.score(X_train_no, Y_train_no) * 100, 2)
acc_knn_no

In [ ]:
knn_normal = KNeighborsClassifier(n_neighbors = 3)
knn_normal.fit(X_train_normal, Y_train_normal)
Y_pred_normal = knn_normal.predict(X_test_normal)
acc_knn_normal = round(knn_normal.score(X_train_normal, Y_train_normal) * 100, 2)
acc_knn_normal

### Gaussian Naive Bayes

In [ ]:
# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

In [ ]:
gaussian_no = GaussianNB()
gaussian_no.fit(X_train_no, Y_train_no)
Y_pred_no = gaussian_no.predict(X_test_no)
acc_gaussian_no = round(gaussian_no.score(X_train_no, Y_train_no) * 100, 2)
acc_gaussian_no

In [ ]:
gaussian_normal = GaussianNB()
gaussian_normal.fit(X_train_normal, Y_train_normal)
Y_pred_normal = gaussian_normal.predict(X_test_normal)
acc_gaussian_normal = round(gaussian_normal.score(X_train_normal, Y_train_normal) * 100, 2)
acc_gaussian_normal

### Perceptron

In [ ]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

In [ ]:
perceptron_no = Perceptron()
perceptron_no.fit(X_train_no, Y_train_no)
Y_pred_no = perceptron_no.predict(X_test_no)
acc_perceptron_no = round(perceptron_no.score(X_train_no, Y_train_no) * 100, 2)
acc_perceptron_no

In [ ]:
perceptron_normal = Perceptron()
perceptron_normal.fit(X_train_normal, Y_train_normal)
Y_pred_normal = perceptron_normal.predict(X_test_normal)
acc_perceptron_normal = round(perceptron_normal.score(X_train_normal, Y_train_normal) * 100, 2)
acc_perceptron_normal

### Linear SVC

In [ ]:
# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
print(acc_linear_svc)

In [ ]:
linear_svc_no = LinearSVC()
linear_svc_no.fit(X_train_no, Y_train_no)
Y_pred_no = linear_svc_no.predict(X_test_no)
acc_linear_svc_no = round(linear_svc_no.score(X_train_no, Y_train_no) * 100, 2)
print(acc_linear_svc_no)

In [ ]:
linear_svc_normal = LinearSVC()
linear_svc_normal.fit(X_train_normal, Y_train_normal)
Y_pred_normal = linear_svc_normal.predict(X_test_normal)
acc_linear_svc_normal = round(linear_svc_normal.score(X_train_normal, Y_train_normal) * 100, 2)
print(acc_linear_svc_normal)

### Stochastic Gradient Descent

In [ ]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

In [ ]:
sgd_no = SGDClassifier()
sgd_no.fit(X_train_no, Y_train_no)
Y_pred_no = sgd_no.predict(X_test_no)
acc_sgd_no = round(sgd_no.score(X_train_no, Y_train_no) * 100, 2)
acc_sgd_no

In [ ]:
sgd_normal = SGDClassifier()
sgd_normal.fit(X_train_normal, Y_train_normal)
Y_pred_normal = sgd_normal.predict(X_test_normal)
acc_sgd_normal = round(sgd_normal.score(X_train_normal, Y_train_normal) * 100, 2)
acc_sgd_normal

### Decision Tree

In [ ]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

In [ ]:
decision_tree_no = DecisionTreeClassifier()
decision_tree_no.fit(X_train_no, Y_train_no)
Y_pred_no = decision_tree_no.predict(X_test_no)
acc_decision_tree_no = round(decision_tree_no.score(X_train_no, Y_train_no) * 100, 2)
acc_decision_tree_no

In [ ]:
decision_tree_normal = DecisionTreeClassifier()
decision_tree_normal.fit(X_train_normal, Y_train_normal)
Y_pred_normal = decision_tree_normal.predict(X_test_normal)
acc_decision_tree_normal = round(decision_tree_normal.score(X_train_normal, Y_train_normal) * 100, 2)
acc_decision_tree_normal

### Random Forest

In [ ]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

In [ ]:
random_forest_no = RandomForestClassifier(n_estimators=100)
random_forest_no.fit(X_train_no, Y_train_no)
Y_pred_no = random_forest_no.predict(X_test_no)
random_forest_no.score(X_train_no, Y_train_no)
acc_random_forest_no = round(random_forest_no.score(X_train_no, Y_train_no) * 100, 2)
acc_random_forest_no

In [ ]:
random_forest_normal = RandomForestClassifier(n_estimators=100)
random_forest_normal.fit(X_train_normal, Y_train_normal)
Y_pred_normal = random_forest_normal.predict(X_test_normal)
random_forest_normal.score(X_train_normal, Y_train_normal)
acc_random_forest_normal = round(random_forest_normal.score(X_train_normal, Y_train_normal) * 100, 2)
acc_random_forest_normal

### 모델 비교

In [ ]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'Support Vector Machines_no', 'Support Vector Machines_normal', 
              'KNN', 'KNN_no', 'KNN_normal', 
              'Logistic Regression', 'Logistic Regression_no', 'Logistic Regression_normal',
              'Random Forest', 'Random Forest_no', 'Random Forest_normal',
              'Naive Bayes', 'Naive Bayes_no', 'Naive Bayes_normal', 
              'Perceptron', 'Perceptron_no', 'Perceptron_normal',
              'Stochastic Gradient Decent', 'Stochastic Gradient Decent_no', 'Stochastic Gradient Decent_normal',
              'Linear SVC', 'Linear SVC_no', 'Linear SVC_normal', 
              'Decision Tree', 'Decision Tree_no', 'Decision Tree_normal'],
    'Score': [acc_svc, acc_svc_no, acc_svc_normal, 
              acc_knn, acc_knn_no, acc_knn_normal, 
              acc_log, acc_log_no, acc_log_normal,
              acc_random_forest, acc_random_forest_no, acc_random_forest_normal, 
              acc_gaussian, acc_gaussian_no, acc_gaussian_normal,
              acc_perceptron, acc_perceptron_no, acc_perceptron_normal,
              acc_sgd, acc_sgd_no, acc_sgd_normal, 
              acc_linear_svc, acc_linear_svc_no, acc_linear_svc_normal, 
              acc_decision_tree, acc_decision_tree_no, acc_decision_tree_normal]})
models.sort_values(by='Score', ascending=False)